In [1]:
import baostock as bs
import pandas as pd
from tqdm import tqdm

In [2]:
lg = bs.login()

login success!


In [3]:
company = pd.read_csv('import/company.csv')
names = list(company['companyId:ID(Company)'])

In [4]:
names[:10]

['000001.SZ',
 '000002.SZ',
 '000004.SZ',
 '000005.SZ',
 '000006.SZ',
 '000007.SZ',
 '000008.SZ',
 '000009.SZ',
 '000010.SZ',
 '000011.SZ']

In [5]:
def reverse_name(name):
    split = name.split('.')
    return '.'.join(reversed(split))
names = [reverse_name(x) for x in names]

In [6]:
names[:10]

['SZ.000001',
 'SZ.000002',
 'SZ.000004',
 'SZ.000005',
 'SZ.000006',
 'SZ.000007',
 'SZ.000008',
 'SZ.000009',
 'SZ.000010',
 'SZ.000011']

In [7]:
drop_col = ['performanceExpStatDate', 'performanceExpUpdateDate', 'performanceExpressROEWa', 'performanceExpressEPSDiluted']
columns = ['companyId', 'ann_date', 'total_asset', 'net_asset', 'change_pct', 'revenue_over_year', 'profit_over_year']

In [11]:
for name in names:
    print(name)
    break

SZ.000001


In [12]:

income_list = []
for name in tqdm(names):
    rs = bs.query_performance_express_report(name)
    assert rs.error_code == '0'
    result_list = []
    while (rs.error_code == '0') & rs.next():
        result_list.append(rs.get_row_data())
        # 获取一条记录，将记录合并在一起
    result = pd.DataFrame(result_list, columns=rs.fields).drop(drop_col, axis=1)
    income_list.append(result)

100%|██████████| 3829/3829 [07:38<00:00,  8.35it/s]


In [13]:
df = pd.concat(income_list)

In [15]:
df.columns = columns

In [16]:
df.head()

,companyId,ann_date,total_asset,net_asset,change_pct,revenue_over_year,profit_over_year
0,sz.000001,2019-01-04,3420753000000.000000,220088000000.000000,0.069231,0.103322,0.068888
1,sz.000001,2020-01-14,3939070000000.000000,273035000000.000000,0.107914,0.181997,0.123325
0,sz.000004,2015-04-16,338282581.120000,80275790.100000,0.057573,0.342357,0.819265
1,sz.000004,2016-04-15,398673507.140000,81523176.250000,-0.030416,0.232865,1.663899
2,sz.000004,2018-04-14,268844295.640000,129389206.960000,-0.365966,-0.518178,-0.829438


In [19]:
df['companyId'] = df.apply(lambda x: reverse_name(x['companyId']), axis=1)

In [20]:
df.head()

,companyId,ann_date,total_asset,net_asset,change_pct,revenue_over_year,profit_over_year
0,000001.sz,2019-01-04,3420753000000.000000,220088000000.000000,0.069231,0.103322,0.068888
1,000001.sz,2020-01-14,3939070000000.000000,273035000000.000000,0.107914,0.181997,0.123325
0,000004.sz,2015-04-16,338282581.120000,80275790.100000,0.057573,0.342357,0.819265
1,000004.sz,2016-04-15,398673507.140000,81523176.250000,-0.030416,0.232865,1.663899
2,000004.sz,2018-04-14,268844295.640000,129389206.960000,-0.365966,-0.518178,-0.829438


In [21]:
df['ann_date'] = pd.to_datetime(df['ann_date'])

In [22]:
df.head()

,companyId,ann_date,total_asset,net_asset,change_pct,revenue_over_year,profit_over_year
0,000001.sz,2019-01-04,3420753000000.000000,220088000000.000000,0.069231,0.103322,0.068888
1,000001.sz,2020-01-14,3939070000000.000000,273035000000.000000,0.107914,0.181997,0.123325
0,000004.sz,2015-04-16,338282581.120000,80275790.100000,0.057573,0.342357,0.819265
1,000004.sz,2016-04-15,398673507.140000,81523176.250000,-0.030416,0.232865,1.663899
2,000004.sz,2018-04-14,268844295.640000,129389206.960000,-0.365966,-0.518178,-0.829438


In [24]:
js = []
for i, row in df.iterrows():
    objs = {
        "measurement": 'report',
        "tags": {
            'companyId': row['companyId']
        },
        "time": row['ann_date'],
        "fields": {
            'total_asset': row['total_asset'],
            'net_asset': row['net_asset'],
            'change_pct': row['change_pct'],
            'revenue_over_year': row['revenue_over_year'],
            'profit_over_year': row['profit_over_year']
        }
    }
    js.append(objs)

In [25]:
len(js)

12202

In [28]:
from influxdb import InfluxDBClient
client = InfluxDBClient('47.101.33.219', 8086, database='stock', username='influx', password='2020liufeng')

In [29]:
client.write_points(js, batch_size=100)

True